In [4]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer

from scipy import sparse
from sklearn.metrics.pairwise import pairwise_distances, cosine_distances, cosine_similarity

In [2]:
rws = pd.read_csv('./movies_data/requests_with_suggestions.csv')

In [3]:
rws

,id,created_utc,title,selftext,suggestions
0,ixcr5j,2020-09-21 19:17:15,Movies from the '70s.,I'm in search of movies from the '70s. Non-En...,tt0120655%tt0069467%tt0069704%tt0450345%tt0066...
1,ix8z2h,2020-09-21 15:46:25,"Of the following mob/mafia movies, which shoul...",\n\nNote that I've already watched all the bi...,tt0901507%tt0221073
2,ix13j8,2020-09-21 09:07:57,What’s your “hidden gem” movie that flew under...,I’ve been through all the main stream movies b...,tt0090315%tt0048491%tt0808417%tt0060176
3,iwz4bu,2020-09-21 07:06:57,Satire like Jojo Rabbit,Really loved the overall story with subtle dar...,tt0436364%tt3464902
4,iwygdm,2020-09-21 06:18:41,English countryside/moors movies,Any good movies set in the beautiful English c...,tt1596365%tt1181614%tt0437179%tt0069050%tt0063...
...,...,...,...,...,...
274,iroswy,2020-09-12 20:05:26,Futuristic Sci-Fi movies,I’m really into movies that take place in the ...,tt0120201%tt0238380%tt0083658%tt0816692%tt3659...
275,iro0jk,2020-09-12 19:16:51,Movies about suicide; and/or a deep hallowing ...,I’m looking for a movie in which the main char...,tt1571249%tt5247022%tt0282698%tt0363589%tt4034...
276,irnvy8,2020-09-12 19:09:27,Cyberpunk/Synthwave/Thriller movies that are n...,"Well I really liked Upgrade/Blade Runner 2049,...",tt1343727%tt1386703%tt3672742%tt0800369%tt6998...
277,irnlz8,2020-09-12 18:53:07,"Hi, I'm looking for hard sci fi movies",Anything that has to do with space and first e...,tt0094737%tt0075860%tt0436364%tt0183523%tt2798...


In [30]:
movie_col = []
text_col = []

for i in rws.index.values:
    sugg_list = rws['suggestions'][i].split('%')
    text = rws['title'][i] + ' ' + rws['selftext'][i]
    
    for sugg in sugg_list:
        movie_col.append(sugg)
        text_col.append(text)

sugg_text_df = pd.DataFrame([movie_col, text_col]).T
sugg_text_df = sugg_text_df.rename(columns = {0:'title', 1:'text'})
sugg_text_df = sugg_text_df.set_index('title')

In [33]:
tf = TfidfVectorizer(max_df= 2000)

In [34]:
tf.fit_transform(sugg_text_df['text'])

<6916x3202 sparse matrix of type '<class 'numpy.float64'>'
	with 240489 stored elements in Compressed Sparse Row format>

In [44]:
sugg_text_df['text']

title
tt0120655    Movies from the '70s. I'm in search of movies ...
tt0069467    Movies from the '70s. I'm in search of movies ...
tt0069704    Movies from the '70s. I'm in search of movies ...
tt0450345    Movies from the '70s. I'm in search of movies ...
tt0066921    Movies from the '70s. I'm in search of movies ...
                                   ...                        
tt0387514    Hi, I'm looking for hard sci fi movies Anythin...
tt0177789    Hi, I'm looking for hard sci fi movies Anythin...
tt0796366    Hi, I'm looking for hard sci fi movies Anythin...
tt0390384    Hi, I'm looking for hard sci fi movies Anythin...
tt1007028    Similar to "The Big Wedding" - Also suggesting...
Name: text, Length: 6916, dtype: object

In [96]:
X = sugg_text_df['text']

X = tf.fit_transform(X)

In [74]:
titles_and_features = pd.DataFrame(X.toarray(), columns = tf.get_feature_names(), index = sugg_text_df.index).drop(columns = ['title'])

In [83]:
titles_and_features = titles_and_features.groupby(by = 'title').sum()

In [144]:
titles_and_features.drop(labels = ' ', inplace = True)

In [145]:
titles_and_features

,00s,0s,10,100,10s,12,13,1408,15,1917,...,youtube,yr7_pfm6zik,yum,zac,zelda,zero,zombie,zombies,zootopia,zorro
title,,,,,,,,,,,,,,,,,,,,,
tt0006864,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
tt0010323,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
tt0011237,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
tt0013086,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
tt0013257,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tt9484998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
tt9541602,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
tt9619798,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [146]:
series = titles_and_features['hilarious'].sort_values(ascending = False)[:10]

for i in series.index:
    print(i, series[i])

tt1245492 0.37006859917704604
tt0383216 0.37006859917704604
tt1190080 0.37006859917704604
tt0071853 0.37006859917704604
tt1341167 0.37006859917704604
tt0383010 0.37006859917704604
tt0094291 0.37006859917704604
tt0942385 0.34721031395780516
tt1386588 0.3460799382836453
tt1396484 0.3270519950488796


In [279]:
test_input = tf.transform(["futuristic"])

test_df = pd.DataFrame(test_input.toarray(), columns = tf.get_feature_names()).T

key_words = test_df.loc[test_df[0] > 0].index.values

rec_dict = {title : 0 for title in titles_and_features.index.values}

for word in key_words:
    series = titles_and_features[word].sort_values(ascending = False)[:10]
    for i in series.index:
        rec_dict[i] += series[i]

score_df = pd.DataFrame(rec_dict, index = ['score']).T

score_df = score_df.loc[score_df['score'] > 0].sort_values(by = 'score', ascending = False)

for i in score_df[:10].index:
    print(title_dict[i])

Alien
Akira
Starship Troopers
Children of Men
Angel's Egg
Equilibrium
Solaris
Ghost in the Shell
Looper
Pacific Rim


In [ ]:
movies = pd.read_csv('./data/movies.tsv', sep = '\t')#.sort_values(by = 'year')

movies = movies[['tconst', 'primaryTitle', 'startYear']]

movies = movies[movies['tconst'].isin(sugg_text_df.index)] #https://stackoverflow.com/a/12098586

movies['startYear'] = movies['startYear'].astype(int)

movies = movies.sort_values(by = 'startYear')

In [232]:
title_dict = movies[['tconst', 'primaryTitle']].set_index('tconst').to_dict()['primaryTitle']